## CNN를 이용한 MNIST 자료 분석

  
과목명 | 금융공학  
교수명 | 안재윤 교수님  
제출일 | 2018년 11월 20일  
  
학　번 | 182STG18  
이　름 | 이하경  

In [2]:
# 필요한 패키지 및 함수, MNIST 데이터 불러오기

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
train_x = np.array([np.reshape(x, (28, 28, 1)) for x in mnist.train.images])
test_x = np.array([np.reshape(x, (28, 28, 1)) for x in mnist.test.images])
train_y = mnist.train.labels
test_y =  mnist.test.labels

In [4]:
# Convolution Layer에 사용하도록 image 데이터의 차원을 4차원으로 변환하여 저장 
# (Nobs, pixel의 H, pixel의 W, input Channel)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(55000, 28, 28, 1) (10000, 28, 28, 1) (55000, 10) (10000, 10)


### Modeling

In [5]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])

# 학습 단계마다 batch size 만큼의 x와 y 데이터를 feed시켜줄 placeholder

#### [Convolution Layer 1] Convolution → Relu → Pooling

In [6]:
# 학습 단계마다 값을 업데이트할 Variable 
# (w의 초기값은 truncated normal random number, bias의 초기값은 0.1 (약간의 양수로 초기화))

w_conv1 = tf.Variable(tf.truncated_normal(shape = [4, 4, 1, 32], stddev = 0.1)) # [FW1, FH1, C, FN1]
b_conv1 = tf.Variable(tf.constant(0.1, shape = [32]))

# 4*4 필터 설정, Input data의 채널 수 1, 필터의 채널을 32개로 확장하여 output

h_conv1 = tf.nn.conv2d(x, w_conv1, strides = [1, 1, 1, 1], padding = 'SAME') 
h_relu1 = tf.nn.relu(h_conv1 + b_conv1)

# 필터를 한칸씩 이동, padding size는 input과 output size가 동일하도록 자동설정함
# x와 weights의 합성곱과 bias을 더한 값을 활성함수 relu에 입력

h_pool1 = tf.nn.max_pool(h_relu1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

# pooling 단계에서 2*2 필터를 2칸씩 이동하며 필터마다 가장 큰 값을 저장하여 
# 첫번째 pooling을 거친 후 차원은 [batch size, 14, 14, 32]

#### [Convolutional Layer 2] Convolution → Relu→ Pooling

In [7]:
w_conv2 = tf.Variable(tf.truncated_normal(shape = [4, 4, 32, 64], stddev = 0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape = [64]))

# Conv. Layer 1과 동일하게 4*4 필터 설정, Layer 2의 Input 채널의 수는 32로 64의 채널로 확장하여 output

h_conv2 = tf.nn.conv2d(h_pool1, w_conv2, strides = [1, 1, 1, 1], padding = 'SAME') 
h_relu2 = tf.nn.relu(h_conv2 + b_conv2)
h_pool2 = tf.nn.max_pool(h_relu2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME') 

# 두번째 pooling을 거친 후의 차원은 [batch size, 7, 7, 64]

아래부터의 과정은 기존의 DNN와 동일하다.
#### [Full-Connected Layer 1] Affine → Relu

In [8]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# Affine layer에 입력하기 위해 2차원으로 변경 (-1은 기존의 mini-batch 차원을 유지해준다)

w_fc1 = tf.Variable(tf.truncated_normal(shape = [7*7*64, 128], stddev = 0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape = [128]))

# node의 개수를 128로 하여 7*7*64개의 특성이 128개의 node와 모두 연결되도록 한다

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

#### [Full-Connected Layer 2] Affine → Relu

In [9]:
# Affine Layer을 한 층 더 추가하였다 (node의 개수는 128개로 동일하게 설정)

w_fc2 = tf.Variable(tf.truncated_normal(shape = [128, 128], stddev = 0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape = [128]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, w_fc2) + b_fc2)

#### [Output Layer] Affine → Softmax (*Final Output!)

In [10]:
w_fc3 = tf.Variable(tf.truncated_normal(shape = [128, 10], stddev = 0.1))              
b_fc3 = tf.Variable(tf.constant(0.1, shape = [10]))
p = tf.nn.softmax(tf.matmul(h_fc2, w_fc3) + b_fc3)                           

# 최종적으로 관측치당 길이 10의 벡터로 출력됨 (관측치 당 10개의 클래스 중 하나로 분류될 확률 값 출력)

학습 단계와 평가 단계에 필요한 함수들을 지정해준다.

In [11]:
cross_entropy = tf.reduce_mean(- tf.reduce_sum(t*tf.log(p), 1))   # 미니배치마다의 -y*log(p)의 합의 평균, 작을 수록 좋음
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy) # Adam Optimizer의 경사하강률 0.01을 이용하여 손실함수를 최소화함
correct_pred = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))         # 분류될 class와 실제 class 값이 같으면 1, 같지 않으면 0
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))      # 전체 데이터 중 몇 개를 정확히 맞추는지 계산

In [19]:
# 학습의 모든 준비가 되었으므로 session을 running하기 전 variable들을 initialize시킴

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 512
epochs = 10 
n_batch = train_x.shape[0] // batch_size

for epoch in range(epochs):
    x_, y_ = shuffle(train_x, train_y)
    
    for i in range(n_batch):
        start = i*batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict = { x:x_[start:end], t:y_[start:end] })
        cost = sess.run(cross_entropy, feed_dict = { x:x_[start:end], t:y_[start:end] })

#        if i % 10 == 0:
#            print(cost)
            
# 총 10번의 학습 단계 동안 512개의 mini batch를 55000 // 512 = 107번 수행해준다
# shuffle 함수를 이용해 train의 x와 y 데이터의 순서를 무작위로 변경

In [20]:
accuracy.eval(session = sess, feed_dict = { x:test_x, t:test_y })

0.9719

In [21]:
print(accuracy.eval(session = sess, feed_dict = { x:train_x[:27500], t:train_y[:27500] }))
print(accuracy.eval(session = sess, feed_dict = { x:train_x[27500:], t:train_y[27500:] })) 

0.9737091
0.97374547
